In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [13]:
llm_config = {
    "model": "gpt-4-turbo",
    "api_key": OPENAI_API_KEY,
}

In [14]:
from autogen.coding import LocalCommandLineCodeExecutor

executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir=".",
)

In [15]:
from autogen import ConversableAgent, AssistantAgent

In [16]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply="Please continue. If everything is done, reply TERMINATE",
)

In [17]:
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
print(code_writer_agent.system_message)

In [18]:
import datetime

today = datetime.datetime.now().date()

message = f"Today is {today}. Create a plot showing gain YDT for NVDA and TSLA. Make sure the code is in markdown code block and save the figure to a file ydt_stock_gains.png"

In [ ]:
chat_result = code_executor_agent.initiate_chat(code_writer_agent, message=message)

In [22]:
def get_stock_prices(stock_symbol, start_date, end_date):
    """
    Get stock prices for a given stock symbol and date range.
    Args:
        stock_symbols (str or list): The stock symbols to get the
        prices for.
        start_date (str): The start date in the format
        'YYYY-MM-DD'.
        end_date (str): The end date in the format 'YYYY-MM-DD'.

    Returns:
        pandas.DataFrame: The stock prices for the given stock
        symbols indexed by date, with one column per stock
        symbol.
    """
    import yfinance as yf

    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    if not stock_data:
        return None
    return stock_data.get("Close")

In [23]:
def plot_stock_prices(stock_prices, filename):
    """Plot the stock prices for the given stock symbols.

    Args:
        stock_prices (pandas.DataFrame): The stock prices for the
        given stock symbols.
    """
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 5))
    for column in stock_prices.columns:
        plt.plot(stock_prices.index, stock_prices[column], label=column)
    plt.title("Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.savefig(filename)
    plt.close()

In [24]:
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir=".",
    functions=[get_stock_prices, plot_stock_prices],
)

In [25]:
code_writer_agent_system_message = executor.format_functions_for_prompt()

In [26]:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    system_message=code_writer_agent_system_message,
)

In [27]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply="Please continue. If everything is done, reply TERMINATE",
)

In [28]:
chat_result = code_executor_agent.initiate_chat(code_writer_agent, message=message)

code_executor_agent (to code_writer_agent):

Today is 2025-01-27. Create a plot showing gain YDT for NVDA and TSLA. Make sure the code is in markdown code block and save the figure to a file ydt_stock_gains.png

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

```python
from functions import get_stock_prices, plot_stock_prices

# Define the stock symbols and the date range (start of the year 2025 to today)
stock_symbols = ['NVDA', 'TSLA']
start_date = '2025-01-01'
end_date = '2025-01-27'  # Today's date as specified

# Get the stock prices using the provided user defined function
stock_prices = get_stock_prices(stock_symbols, start_date, end_date)

# Calculate the Year-To-Date (YTD) gains by comparing the latest price to the first price of the year
ytd_gains = stock_prices.ffill().iloc[-1] / stock_prices.ffill().iloc[0] - 1
ytd_gains.index.name = 'Stock Symbol'
ytd_gains.name = 'YDT Gain'

# Plotting the YDT g